# 미니배치 학습 

In [1]:
import sys, os 
sys.path.append( os.pardir )          # 부모 디렉토리 경로를 sys.path에 추가(p.98)
import numpy as np 
from dataset.mnist import load_mnist   # MNIST 데이터를 Numpy 객체로 불러오기 

In [2]:
(x_train, t_train), (x_test, t_test) = load_mnist( normalize=True, one_hot_label=True)

print(x_train.shape)    # (60000, 784) = (데이터 개수, 평활화된 이미지 배열 28x28 -> 784)
print(t_train.shape)    # (60000, 10) = (데이터 개수, 레이블 클래스 개수)

(60000, 784)
(60000, 10)


In [3]:
t_train[0]    # 0 번째 데이터의 레이블  
              # '정답은 5 in one-hot encoding'

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

### 미니배치 고르기 

In [4]:
train_size  =  x_train.shape[0]   # 전체 데이터 개수(전체 batch) 
batch_size  =  10                 # 모집단 (mini-batch)

batch_mask  =  np.random.choice(train_size, batch_size) 
print(batch_mask)                 # 0 ~ 59999 숫자 중 임의로 10개 고름  

[57572 14945 45645 39530 18586  5334 10042 30228 40818  1034]


In [5]:
x_batch   =  x_train[batch_mask]  # 뽑힌 숫자를 index로 사용 
t_batch   =  t_train[batch_mask]

print(x_batch.shape)
print(t_batch.shape)

(10, 784)
(10, 10)


In [6]:
np.random.choice(60000, 10)   #[0, 60000) 수 중에서 무작위로 10개 뽑아라 

array([55489, 19539, 24795, 34910, 47593, 56433, 53601, 28763, 29413,
       56443])

## (batch용) 교차 엔트로피 오차 구현하기 

In [7]:
def CEE_batch(y, t):   # y: output, t: label 
    if y.ndim == 1:    # y객체가 1-D 배열이냐? => batch_size = 1
        t  = t.reshape(1, t.size)  # 1-D 배열로 변환 
        y  = y.reshape(1, y.size)  # 1-D 배열로 변환 
        
    epsilon     =  1e-7    
    batch_size  =  y.shape[0]  # 총 데이터 개수 N 
    return -np.sum(t * np.log(y + epsilon)) / batch_size  # 평균 CEE = sum(CEE)/N 

### batch size = 2인 데이터 예시 

In [8]:
y  =  [[0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0],
        [0.2, 0.3, 0.0, 0.0, 0.6, 0.0, 0.1, 0.0, 0.0, 0.0]  ] # 숫자 클래스 10개

t  =  [[ 0 ,  0  ,  1 ,  0 ,   0 ,  0 ,  0 ,  0 ,  0 ,  0 ],
       [ 0,   1,    0 ,  0 ,   0 ,  0 ,  0 ,  0 ,  0  , 0]] # one-hot encoding 

In [9]:
result_y   =  np.array(y)      # batch_size = 2 
label_t    =  np.array(t)       

print(result_y.shape)
print(label_t.shape)


(2, 10)
(2, 10)


In [10]:
loss_function   =  CEE_batch(result_y, label_t)
print("loss_function = ", loss_function)

loss_function =  0.8573989640459981


## (batch용) 교차 엔트로피 오차 for non one-hot encoding 

In [11]:
(x_train_non, t_train_non), (x_test_non, t_test_non) = load_mnist( normalize=True, one_hot_label=False)

print(x_train_non.shape)    # (60000, 784) = (데이터 개수, 평활화된 이미지 배열 28x28 -> 784)
print(t_train_non.shape)    # (60000, ) = (데이터 개수, )

(60000, 784)
(60000,)


In [12]:
t_train[0]    # 0 번째 데이터의 레이블  
              # '정답은 5 in one-hot encoding'

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [13]:
t_train_non[0]   # non one-hot encoding 

5

In [14]:
def CEE_batch_non(y, t):     #y: output,  t: lable 
    if y.ndim == 1:       # yrorcprk 1D 배열 => batch_size = 1 
        t  =  t.reshape(1, t.size) 
        y  =  y.reshape(1, y.size) 
        
    epsilon     = 1e-7
    batch_size  = y.shape[0]  # 총 데이터 개수N 
    return  -np.sum(np.log( y[np.arange(batch_size), t] + epsilon  ))/batch_size # 평균 CEE = sum(CEE) / N 

### batch size = 2인 데이터 예시 

In [15]:
y  =  [[0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0],
        [0.2, 0.3, 0.0, 0.0, 0.6, 0.0, 0.1, 0.0, 0.0, 0.0]  ] # 숫자 클래스 10개

t  =  [2, 1] # Non one-hot encoding
             # 0번째 데이터 t[0]의 정답 = '2' 
             # 1번째 데이터 t[1]의 정답 = '1'

In [16]:
result_y   =  np.array(y)      # batch_size = 2 
label_t    =  np.array(t)       

print(result_y.shape)
print(label_t.shape)

(2, 10)
(2,)


In [17]:
loss_function   =  CEE_batch_non(result_y, label_t)
print("loss_function = ", loss_function)

loss_function =  0.8573989640459981


#### Array slicing 예시 

In [18]:
np.arange(2)   

array([0, 1])

In [19]:
label_t

array([2, 1])

In [20]:
result_y


array([[0.1 , 0.05, 0.6 , 0.  , 0.05, 0.1 , 0.  , 0.1 , 0.  , 0.  ],
       [0.2 , 0.3 , 0.  , 0.  , 0.6 , 0.  , 0.1 , 0.  , 0.  , 0.  ]])

In [21]:
result_y[ np.arange(2), label_t ]      # Array slicing 

array([0.6, 0.3])

In [22]:
result_y[ [0,1], [2,1]]   # 좌표 (0, 2) ->  0.6
                          # 좌표 (1, 1) ->  0.3 

array([0.6, 0.3])